In [7]:
import pandas as pd

# load results - datacite, scholex and overton

# nerc_citations_2025/results/latest_results_dataCite.csv
# /home/matnic/Projects/citationNotebook/nerc_citations_2025/results/latest_results_dataCite.csv
# dataCite_df = pd.read_csv("results/latest_results_dataCite.csv")
dataCite_df = pd.read_csv("results/latest_results_dataCite_publicationInfo.csv") # change datacite to write to just latest_results file
scholex_df = pd.read_csv("results/latest_results_scholex.csv")
overton_df = pd.read_csv("results/latest_results_overton.csv")
# merge in a general way

In [ ]:
with pd.option_context('display.max_colwidth', None):
    print(dataCite_df.iloc[1])

In [ ]:
with pd.option_context('display.max_colwidth', None):
    print(scholex_df.iloc[205])

In [4]:
with pd.option_context('display.max_colwidth', None):
    print(overton_df.iloc[51])

data_doi                                             10.5285/76a29c5b55204b66a40308fc2ba9cdb3
data_publisher                                  Centre for Environmental Data Analysis (CEDA)
data_title                 GloboLakes: Lake Surface Water Temperature (LSWT) v4.0 (1995-2016)
data_publication_year                                                                  2019.0
data_authors                                    ['Carrea, Laura', 'Merchant, Christopher J.']
relation_type                                                                      references
pub_doi                  https://www.ipcc.ch/report/sixth-assessment-report-working-group-ii/
pub_title                      AR6 Climate Change 2022: Impacts, Adaptation and Vulnerability
pub_date                                                                           2022-02-01
pub_authors                                                                          ['IPCC']
pub_type                                                    

In [11]:
# take data_doi and pub_doi columns from each df, concatenate vertically
doi_df = pd.concat([scholex_df[['data_doi', 'pub_doi']], dataCite_df[['data_doi', 'pub_doi']], overton_df[['data_doi', 'pub_doi']]], ignore_index=True)
doi_df_unique = doi_df.drop_duplicates(subset=['data_doi', 'pub_doi'])
print(len(doi_df))
print(len(doi_df_unique))

24920
13654


In [12]:
# if all sources are wanted, 
print(dataCite_df.columns)

Index(['data_doi', 'data_publisher', 'data_title', 'data_publication_year',
       'data_authors', 'relation_type', 'pub_doi', 'pub_title', 'pub_date',
       'pub_authors', 'source_id', 'pub_publisher', 'pub_type'],
      dtype='object')


In [15]:
df_list = [dataCite_df, scholex_df, overton_df]
def merge_data_citations_dfs(df_list):

    combined_df = pd.concat(df_list, ignore_index=True)

    deduped_df = combined_df.drop_duplicates(subset=list(("data_doi", "pub_doi")), keep="first")

    return deduped_df

In [16]:
deduped_df = merge_data_citations_dfs(df_list)

In [17]:
len(deduped_df)

13654

In [22]:
deduped_df["pub_type"].value_counts()


pub_type
dataset                8729
literature             2281
journal-article        1436
Info not given          906
not a doi               113
Publication             112
posted-content           59
book-chapter              8
Working paper             6
proceedings-article       1
report                    1
other                     1
API request failed        1
Name: count, dtype: int64